In [5]:
from lyricsgenius import Genius
from bs4 import BeautifulSoup
import requests
import pandas as pd
from itertools import islice
import os
import re

# Get top 50 rap songs between 2018 and 2022 from billboard website

In [6]:
def get_top50_from_url(year):
    #example = url_start + '2018' + url_end
    url_start = 'https://www.billboard.com/charts/year-end/'
    url_end = '/hot-rap-songs/'

    url = url_start + str(year) + url_end
    response = requests.get(url)

    soup = BeautifulSoup(response.text, 'html.parser')
    results = soup.find_all('div', {'class': 'o-chart-results-list-row-container'})

    data = []

    for i in range(len(results)):
        result = results[i]
        title = result.find('h3').text.strip()
        spans = result.find_all('span')
        artist = spans[1].text.strip()
        data.append({'position': i+1, 'title': title, 'artist': artist, 'year': year})

    df = pd.DataFrame(data)
    return df


In [7]:
# Get top 50 songs for each year
df = pd.DataFrame(columns=['position', 'title', 'artist', 'year'])
for i in range(2018,2023):
    df = df.append(get_top50_from_url(i), ignore_index=True)

df.head()

C:\Users\Surface\AppData\Local\Temp\ipykernel_17052\3513559878.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(get_top50_from_url(i), ignore_index=True)
C:\Users\Surface\AppData\Local\Temp\ipykernel_17052\3513559878.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(get_top50_from_url(i), ignore_index=True)
C:\Users\Surface\AppData\Local\Temp\ipykernel_17052\3513559878.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(get_top50_from_url(i), ignore_index=True)
C:\Users\Surface\AppData\Local\Temp\ipykernel_17052\3513559878.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.appen

,position,title,artist,year
0,1,God's Plan,Drake,2018
1,2,I Like It,"Cardi B, Bad Bunny & J Balvin",2018
2,3,In My Feelings,Drake,2018
3,4,Psycho,Post Malone Featuring Ty Dolla $ign,2018
4,5,Nice For What,Drake,2018


In [8]:
# write to csv
df.to_csv('top50.csv', index=False)

# Use lyricsmaster to get lyrics for each song from Genius.com API

In [17]:
def remove_special_chars(file_name):
    # Define special characters to remove
    special_chars = "!@#$%^*()[]{};:/<>?\|`~-=_+’"

    # Remove special characters from file name
    new_file_name = ''.join(c for c in file_name if c not in special_chars)

    return new_file_name

In [40]:
# write a 0 to the index.txt file
with open('index.txt', 'w') as f:
    f.write('0')

i = 0

In [41]:
def scrape_lyrics():
    # Authenticate with Genius API
    genius = Genius('uWk7jiPtIjLiomyfQS1Y6QvB_Aw3EWQr3ODfulXHBKZU1kBWFBzNwJMQ0rbi3Ig6')
    # create the subdirectory /songs/ if it does not exist
    if not os.path.exists('songs'):
        os.makedirs('songs')
    # read the index.txt file to get the index of the last song that was written to a json file
    with open('index.txt', 'r') as f:
        i = int(f.read())

    print(i)
    # iterate through each row in the dataframe starting at index i
    for row,column in islice(df.iterrows(), i, None):
        song = genius.search_song(column['title'], column['artist'])
        # write the song object to a json file named by the song title and artist name in the folder songs
        file_name = 'song' + str(i) + '.json'
        # modify the file name to remove all special characters that are not allowed in a file name
        file_name = remove_special_chars(file_name)
        song.save_lyrics(file_name, overwrite=True)
        # move the created json into the /songs/ folder
        os.rename(os.getcwd() + '\\' + file_name, os.getcwd() + '\\' + 'songs' + '\\' + file_name)
        # write a txt file with the content i and the name index.txt
        i += 1
        with open('index.txt', 'w') as f:
            f.write(str(i))
        df.loc[row, 'lyrics'] = song.lyrics

In [42]:
while i < len(df):
    try:
        scrape_lyrics()
    except requests.exceptions.Timeout:
        print('Request timed out')
    except requests.exceptions.RequestException as e:
        print('Error:', e)

# delete the index.txt file
os.remove('index.txt')

0
Searching for "God's Plan" by Drake...


KeyboardInterrupt: 

## Read the lyrics from the json files and add them to the dataframe

In [13]:
import json
from glob import glob

In [14]:
# get a list of all the json files in the /songs/ folder
json_files = glob('songs/*.json')

In [15]:
# read each json file and add the lyrics to the dataframe
ies = []
for file in json_files:
    with open(file, 'r') as f:
        data = json.load(f)
    # extract the i value from the file name
    i = int(''.join(c for c in file if c.isdigit()))
    ies.append(i)
    print(i)
    # add the lyrics to the dataframe
    df.loc[i, 'lyrics'] = data['lyrics']

In [16]:
# write the dataframe to a csv file
df.to_csv('top50.csv', index=False)

In [16]:
'song1.json' -> 1
'song12.json' -> 12
'song123.json' -> 123